In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import nn
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as T
from PIL import Image
import genericpath

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# force working on cpu due to memory limitation
#device = torch.device("cpu")
#tpr = torch.hub.load('yangsenius/TransPose:main', 'tpr_a4_256x192', pretrained=True)
#tph = torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True, device=device)


#print(tph)
DATASET_PATH = './dataset/'
positions = os.listdir(DATASET_PATH)
#print(os.listdir(DATASET_PATH+entries[0]))
images = list()


def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result


class YogaPoseDataset(Dataset):

    def __init__(self, dataset_path, size=(256, 192), transform=None):
        self.data_path = dataset_path
        self.size = size
        self.transform = transform

        # call to init the data
        self._init_data()

    def _init_data(self):
        images = list()

        for _, directory_class in enumerate(os.listdir(self.data_path)):
            class_path = os.path.join(self.data_path, directory_class)
            for file_name in os.listdir(class_path):
                f = cv2.imread(os.path.join(class_path, file_name), cv2.IMREAD_COLOR)
                f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)

                if self.transform is not None:
                    f = self.transform(f)

                data = torch.reshape(torch.FloatTensor(f).to(device), (3, self.size[0], self.size[1]))

                # format example  images[x][0] -> (label, input)
                # format example  images[x][1] -> [other information]
                # images[x] -> ((class_id, image_tensor), [filename])
                images.append((int(directory_class), data))

        np.random.shuffle(images)
        self.images = images

    def __len__(self):
        # returns the number of samples in our dataset
        return len(self.images)

    def getData(self):
        return self.images

    def __getitem__(self, idx):
        """

        Args:
            idx: the index of the sample

        Returns: a tuple (class, input) for the given sample

        """
        return self.images[idx]

    def getFileName(self, idx):
        return str(self.images[idx][1][0])

    def getOriginalImage(self, idx):
        class_path = os.path.join(self.data_path, str(self.images[idx][0][0]))
        out = cv2.imread(os.path.join(class_path, self.getFileName(idx)), cv2.IMREAD_COLOR)
        print(self.getFileName(idx))
        return out

    def collate_fn(self, data):
        #print(data)
        Xs = torch.stack([x[1] for x in data])
        y = torch.stack([torch.tensor(x[0]) for x in data])
        return Xs, y

In [2]:


DATASET_PATH = './data/images/'
ANNOTATION_PATH = './data/annotations/'
MODEL_NAME = "tpr_a4_256x192"
norm_transform = T.Compose([T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), ])
dataset = YogaPoseDataset(DATASET_PATH, transform=norm_transform)

In [3]:
# get model from torch hub
assert MODEL_NAME in ["tpr_a4_256x192", "tph_a4_256x192"]
modelyaml = {
    "tph_a4_256x192": "models_yaml/TP_H_w48_256x192_stage3_1_4_d96_h192_relu_enc4_mh1.yaml",
    "tpr_a4_256x192": "models_yaml/TP_R_256x192_d256_h1024_enc4_mh8.yaml"
}
model = torch.hub.load('yangsenius/TransPose:main', MODEL_NAME, pretrained=True, force_reload=True, verbose=2)
model.to(device)


Downloading: "https://github.com/yangsenius/TransPose/archive/main.zip" to /home/michele/.cache/torch/hub/main.zip


>>Load pretrained weights from url: https://github.com/yangsenius/TransPose/releases/download/Hub/tp_r_256x192_enc4_d256_h1024_mh8.pth
Successfully loaded model  (on cpu) with pretrained weights!


/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_h.py:676: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or self.pretrained_layers[0] is '*':
/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_r.py:333: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature ** (2 * (dim_t // 2) / one_direction_feats)


TransPoseR(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=

In [4]:
from TransPose.lib.config import cfg
from TransPose.lib.utils import transforms
from TransPose.lib.core.inference import get_final_preds
#from TransPose.lib.utils import transforms
from TransPose.visualize import inspect_atten_map_by_locations

In [5]:

split_position = (len(dataset) // 10) * 7
trainset = dataset[:split_position]
testset = dataset[split_position:]
#trainset = dataset[:100]
#print(trainset)

In [6]:
print(len(trainset))
print(split_position)
print(len(dataset))
dataset[3]
trainset[2]

4193
4193
5992


(97,
 tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [ 2.0434,  2.0434,  2.0263,  ...,  2.0434,  2.0434,  2.0434],
          [ 2.0434,  2.0434,  2.0263,  ...,  2.0434,  2.0263,  2.0263],
          [ 2.0263,  2.0263,  2.0092,  ...,  2.0434,  2.0263,  2.0263]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [ 2.1660,  2.1660,  2.1485,  ...,  2.1660,  2.1660,  2.1660],
          [ 2.1660,  2.1660,  2.1485,  ...,  2.1660,  2.1485,  2.1485],
          [ 2.1485,  2.1485,  2.1310,  ...,  2.1660,  2.1485,  2.1485]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.

In [7]:
OUT_DIR = "./out/"
idx = 0

if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)

In [8]:
'''
with torch.no_grad():
	model.eval()
	img = dataset[idx][1]

	inputs = torch.cat([img.to(device)]).unsqueeze(0)
	outputs = model(inputs)

	if isinstance(outputs, list):
		output = outputs[-1]
	else:
		output = outputs

	if cfg.TEST.FLIP_TEST:
		input_flipped = np.flip(inputs.cpu().numpy(), 3).copy()
		input_flipped = torch.from_numpy(input_flipped).cuda()
		outputs_flipped = model(input_flipped)

		if isinstance(outputs_flipped, list):
			output_flipped = outputs_flipped[-1]
		else:
			output_flipped = outputs_flipped

		output_flipped = transforms.flip_back(output_flipped.cpu().numpy(), dataset.flip_pairs)
		output_flipped = torch.from_numpy(output_flipped.copy()).cuda()

		output = (output + output_flipped) * 0.5

	preds, maxvals = get_final_preds(cfg, output.clone().cpu().numpy(), None, None, transform_back=False)

# from heatmap_coord to original_image_coord
query_locations = np.array([p * 4 + 0.5 for p in preds[0]])

inspect_atten_map_by_locations(img, model, query_locations, model_name="transposer", mode='dependency', save_img=True, threshold=0.0, outinfo=(OUT_DIR, dataset.getFileName(idx)))

cv2.imwrite(OUT_DIR + dataset.getFileName(idx) + "_original_img.jpg", dataset.getOriginalImage(idx))
'''


'\nwith torch.no_grad():\n\tmodel.eval()\n\timg = dataset[idx][1]\n\n\tinputs = torch.cat([img.to(device)]).unsqueeze(0)\n\toutputs = model(inputs)\n\n\tif isinstance(outputs, list):\n\t\toutput = outputs[-1]\n\telse:\n\t\toutput = outputs\n\n\tif cfg.TEST.FLIP_TEST:\n\t\tinput_flipped = np.flip(inputs.cpu().numpy(), 3).copy()\n\t\tinput_flipped = torch.from_numpy(input_flipped).cuda()\n\t\toutputs_flipped = model(input_flipped)\n\n\t\tif isinstance(outputs_flipped, list):\n\t\t\toutput_flipped = outputs_flipped[-1]\n\t\telse:\n\t\t\toutput_flipped = outputs_flipped\n\n\t\toutput_flipped = transforms.flip_back(output_flipped.cpu().numpy(), dataset.flip_pairs)\n\t\toutput_flipped = torch.from_numpy(output_flipped.copy()).cuda()\n\n\t\toutput = (output + output_flipped) * 0.5\n\n\tpreds, maxvals = get_final_preds(cfg, output.clone().cpu().numpy(), None, None, transform_back=False)\n\n# from heatmap_coord to original_image_coord\nquery_locations = np.array([p * 4 + 0.5 for p in preds[0]])

In [9]:
class PoseClassifier(nn.Module):
    def __init__(self, n_class,
                 transpose_model=torch.hub.load('yangsenius/TransPose:main', 'tph_a4_256x192', pretrained=True,
                                                device=device), fine_tune=False, pretrained=True):
        super(PoseClassifier, self).__init__()
        layers = []
        dropout = 0
        hidden_layers = [128, 512, 512, 512, 512, 128]
        self.tph = transpose_model
        layers.append(nn.Conv2d(17, 128, 3, padding=1))
        #self.pool1 = nn.MaxPool2d((2, 2), 2)
        for index, value in enumerate(hidden_layers[:-1]):
            layers += [nn.MaxPool2d((2, 2), 2), nn.ReLU()]
            layers += [nn.Dropout(dropout)]
            layers += [nn.Conv2d(hidden_layers[index], hidden_layers[index + 1], 3, padding=1)]
        #self.fc1 = nn.Linear(52224,10000).to(device)
        #self.fc2 = nn.Linear(10000,1000).to(device)
        #self.fc3 = nn.Linear(1000,n_class).to(device)
        self.relu = nn.ReLU()
        layers += [nn.MaxPool2d((2, 2), 2), nn.Flatten(), nn.ReLU()]
        #layers += [nn.Linear(hidden_layers[-1], num_classes)]
        layers += [nn.Dropout(dropout)]
        self.layers = nn.Sequential(*layers)

        #'''
        self.conv1 = nn.Conv2d(17, hidden_layers[0], 3, padding=1)
        self.pool1 = nn.MaxPool2d((2, 2), 2)
        self.conv2 = nn.Conv2d(hidden_layers[0], hidden_layers[1], 3, padding=1)
        self.conv3 = nn.Conv2d(hidden_layers[1], hidden_layers[2], 3, padding=1)
        self.conv4 = nn.Conv2d(hidden_layers[2], hidden_layers[3], 3, padding=1)
        self.conv5 = nn.Conv2d(hidden_layers[3], hidden_layers[4], 3, padding=1)
        self.conv6 = nn.Conv2d(hidden_layers[4], hidden_layers[5], 3, padding=1)
        #'''

        self.flatten = nn.Flatten()
        self.classifier = nn.Linear(hidden_layers[-1]*2,n_class)


    def forward(self, x):
        out = self.tph(x)
        #print(out.size(),"AFTER TPH")
        out = self.conv1(out)
        #print(out.size(),"AFTER CONV1")
        out = self.pool1(out)
        #print(out.size(),"AFTER POOL")
        out = self.conv2(out)
        #print(out.size(),"AFTER CONV2")
        out = self.pool1(out)
        #print(out.size(),"AFTER POOL")
        out = self.conv3(out)
        #print(out.size(),"AFTER CONV3")
        out = self.pool1(out)
        #print(out.size(),"AFTER POOL")
        out = self.conv4(out)
        #print(out.size(),"AFTER CONV4")
        out = self.pool1(out)
        #print(out.size(),"AFTER POOL")
        out = self.conv5(out)
        #print(out.size(),"AFTER CONV5")
        out = self.pool1(out)
        #print(out.size(),"AFTER POOL")
        out = self.conv6(out)
        #print(out.size(),"AFTER CONV7")
        out = self.flatten(out)

        #print(out.size(),"AFTER FLATTEN")
        out = self.classifier(out)
        return out
        #return torch.softmax(out, dim=1)

    '''
    x = x.to(device)
        print(x.size(),"INPUT")
        out = self.tph(x)
        #out = self.relu(out)
        print(out.size(), "AFTER TPR")
        #out = torch.einsum("abcd -> abc",out)
        #print(out.size())
        out = self.conv1(out)
        print(out.size())
        out = self.pool1(out)
        out = self.relu(out)
        print(out.size(),'AFTER POOLING')
        out = torch.reshape(out,(out.size(0),-1))
        #print(out.size())
        out = self.fc1(out)
        #m = nn.BatchNorm1d(1000,device=device)
        #out = m(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
    '''




Using cache found in /home/michele/.cache/torch/hub/yangsenius_TransPose_main


>>Load pretrained weights from url: https://github.com/yangsenius/TransPose/releases/download/Hub/tp_h_48_256x192_enc4_d96_h192_mh1.pth


/home/michele/.cache/torch/hub/yangsenius_TransPose_main/lib/models/transpose_h.py:514: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature ** (2 * (dim_t // 2) / one_direction_feats)


Successfully loaded model  (on cpu) with pretrained weights!


In [10]:
num_classes = 107
model = PoseClassifier(n_class=num_classes, transpose_model=model)
num_epochs = 50
batch_size = 16
learning_rate = 1e-4
learning_rate_decay = 0.99
params_to_update = model.parameters()


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
model.to(device)
fine_tune = True
if fine_tune:
    params_to_update = []
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    for param in model.tph.parameters():
        param.requires_grad = False
    for p in model.parameters():
        if p.requires_grad == True:
            params_to_update.append(p)
else:
    params_to_update = model.parameters()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=learning_rate)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=False,
                                           collate_fn=dataset.collate_fn)
val_loader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False,collate_fn=dataset.collate_fn)
#val_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=False, collate_fn=dataset.collate_fn)
# Train the model
lr = learning_rate
total_step = len(train_loader)
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []

In [12]:

for item in train_loader:
    print(item)


(tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-0.3369, -0.3198, -0.3198,  ..., -0.9534, -0.8849, -0.9192],
          [ 0.0227,  0.0227,  0.0227,  ..., -0.9020, -0.9877, -0.8507],
          [ 1.7352,  1.7352,  1.7352,  ..., -1.4329, -1.5014, -1.5014]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-0.2150, -0.1975, -0.1975,  ..., -0.8452, -0.7752, -0.8102],
          [ 0.1527,  0.1527,  0.1527,  ..., -0.7927, -0.8803, -0.7402],
          [ 1.9034,  1.9034,  1.9034,  ..., -1.3354, -1.4055, -1.4055]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 

In [ ]:
model.to(device)
params_to_update = model.parameters()
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=learning_rate)
#train_loader = torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=False)
#val_loader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False)
# Train the model
lr = learning_rate
total_step = len(train_loader)
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []
#best_model = type(model)(num_classes, fine_tune, pretrained) # get a new instance
for epoch in range(num_epochs):

    model.train()
    correct = 0
    total = 0
    loss_iter = 0

    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(outputs,labels)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_iter += loss.item()


        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

    accuracy = 100 * correct / total

    print('Training accuracy is: {} %'.format(accuracy))
    loss_train.append(loss_iter / (len(train_loader) * batch_size))

    # Code to update the lr
    lr *= learning_rate_decay
    update_lr(optimizer, lr)

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        loss_iter = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            #print(outputs)
            #print(torch.sum(outputs[0]),torch.sum(outputs[1]))
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            #print(labels)
            total += labels.size(0)
            #print("PREDICTED",predicted)
            #print("LABELS",labels)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss_iter += loss.item()

        loss_val.append(loss_iter / (len(val_loader) * batch_size))

        accuracy = 100 * correct / total
        accuracy_val.append(accuracy)

        print('Validataion accuracy is: {} %'.format(accuracy))
        early_stop = False
        patience = 3
        if epoch > patience - 1:
            for j in range(patience - 1):
                if max(accuracy_val) > list(reversed(accuracy_val))[j]:
                    if "not_improving_epochs" in locals():
                        not_improving_epochs += 1
                    else:
                        not_improving_epochs = 1
                    print('Not saving the model')
                else:
                    not_improving_epochs = 0
                    best_model = model
                    print("Saving the model")
                    break
                if not_improving_epochs >= patience:
                    early_stop = True
                    print('Early stopping')
                    break
                break

plt.figure(2)
plt.plot(loss_train, 'r', label='Train loss')
plt.plot(loss_val, 'g', label='Val loss')
plt.legend()
plt.show()

plt.figure(3)
plt.plot(accuracy_val, 'r', label='Val accuracy')
plt.legend()
plt.show()

Epoch [1/50], Step [100/263], Loss: 4.2095
Epoch [1/50], Step [200/263], Loss: 3.7386
Training accuracy is: 6.749344145003578 %
Validataion accuracy is: 14.785992217898833 %
Epoch [2/50], Step [100/263], Loss: 2.9295
Epoch [2/50], Step [200/263], Loss: 3.1113
Training accuracy is: 22.537562604340568 %
Validataion accuracy is: 30.405780989438576 %
Epoch [3/50], Step [100/263], Loss: 2.4707
Epoch [3/50], Step [200/263], Loss: 2.2721
Training accuracy is: 39.49439542093966 %
Validataion accuracy is: 41.07837687604225 %
Epoch [4/50], Step [100/263], Loss: 1.6721
Epoch [4/50], Step [200/263], Loss: 1.6905
Training accuracy is: 51.80062008108753 %
Validataion accuracy is: 46.41467481934408 %
Saving the model
Epoch [5/50], Step [100/263], Loss: 1.2321
Epoch [5/50], Step [200/263], Loss: 1.3247
